In [1]:
#####################################################
# <실습5>
# 예제7-9 챗봇 학습 데이터 불러오기 (엑셀에서 db로)
# /train_tools/qna/load_train_data.py
#####################################################
import sys
sys.path.append('C:/chatbot')

In [2]:
import pymysql
import openpyxl

from config.DatabaseConfig import *   # DB접속 정보 불러오기

In [18]:
## DB 학습데이터 모두 삭제 (초기화)
def all_clear_train_data(db):
    # 기존 데이터 삭제
    sql = '''
            DELETE FROM chatbot_train_data    
    '''
    with db.cursor() as cursor:
        cursor.execute(sql)
        
# DB에 데이터 저장
def insert_data(db, xls_row):
    intent, ner, query, answer, answer_image = xls_row
    
    sql = '''
        INSERT chatbot_train_data (
                intent, ner, query, answer, answer_image)
        VALUES ('%s', '%s', '%s', '%s', '%s')
    ''' % (intent.value, ner.value, query.value, 
                           answer.value, answer_image.value)
    
    # 엑셀에서 불러온 cell에 데이터가 없는 경우 null로 치환 (엑셀과 DB의 차이)
    sql = sql.replace("'None'", "null")
    print(sql)

    with db.cursor() as cursor:
        cursor.execute(sql)
        print('{} 저장'.format(query.value))
        db.commit()
    print('-'*80)    
    

In [19]:
# [참고:엑셀] 학습용 엑셀 파일 읽어오기
train_file = './train_data.xlsx'
wb = openpyxl.load_workbook(train_file)
sheet = wb['Sheet1']
for row in sheet.iter_rows(min_row=2): # 1:헤더 skip, 최소 2에서 시작
    print(row)

(<Cell 'Sheet1'.A2>, <Cell 'Sheet1'.B2>, <Cell 'Sheet1'.C2>, <Cell 'Sheet1'.D2>, <Cell 'Sheet1'.E2>)
(<Cell 'Sheet1'.A3>, <Cell 'Sheet1'.B3>, <Cell 'Sheet1'.C3>, <Cell 'Sheet1'.D3>, <Cell 'Sheet1'.E3>)
(<Cell 'Sheet1'.A4>, <Cell 'Sheet1'.B4>, <Cell 'Sheet1'.C4>, <Cell 'Sheet1'.D4>, <Cell 'Sheet1'.E4>)
(<Cell 'Sheet1'.A5>, <Cell 'Sheet1'.B5>, <Cell 'Sheet1'.C5>, <Cell 'Sheet1'.D5>, <Cell 'Sheet1'.E5>)
(<Cell 'Sheet1'.A6>, <Cell 'Sheet1'.B6>, <Cell 'Sheet1'.C6>, <Cell 'Sheet1'.D6>, <Cell 'Sheet1'.E6>)
(<Cell 'Sheet1'.A7>, <Cell 'Sheet1'.B7>, <Cell 'Sheet1'.C7>, <Cell 'Sheet1'.D7>, <Cell 'Sheet1'.E7>)


In [20]:
row[3].value

'욕하면 나빠요 ㅠ'

In [21]:
db = None

try:
    db = pymysql.connect(
        host = DB_HOST,
        user = DB_USER,
        passwd = DB_PASSWORD,
        db = DB_NAME,
        charset = 'utf8'
    )
    
    # 기존 DB의 테이블 내용 삭제 (초기화)
    all_clear_train_data(db)
    
    # 학습용 엑셀 파일 불러와서 DB에 Insert하기
    train_file = './train_data.xlsx'
    wb = openpyxl.load_workbook(train_file)
    sheet = wb['Sheet1']
    for row in sheet.iter_rows(min_row=2): # 1:헤더 skip, 최소 2에서 시작
        insert_data(db, row)
        
    wb.close()

except Exception as e:
    print(e)
        
finally:
    if db is not None:
        db.close()


        INSERT chatbot_train_data (
                intent, ner, query, answer, answer_image)
        VALUES ('인사', null, '안녕하세요', '네 안녕하세요 :D
반갑습니다. 저는 챗봇입니다.', 'https://i.imgur.com/UluUFMp.jpg')
    
안녕하세요 저장
--------------------------------------------------------------------------------

        INSERT chatbot_train_data (
                intent, ner, query, answer, answer_image)
        VALUES ('인사', null, '반가워요', '네 안녕하세요 :D
반갑습니다. 저는 챗봇입니다.', 'https://i.imgur.com/UluUFMp.jpg')
    
반가워요 저장
--------------------------------------------------------------------------------

        INSERT chatbot_train_data (
                intent, ner, query, answer, answer_image)
        VALUES ('주문', 'B_FOOD', '{B_FOOD} 주문할게요', '{B_FOOD} 주문 처리 완료되었습니다. 
주문해주셔서 감사합니다.', null)
    
{B_FOOD} 주문할게요 저장
--------------------------------------------------------------------------------

        INSERT chatbot_train_data (
                intent, ner, query, answer, answer_image)
        VALUES ('주문', 'B